## Import Dependencies 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import lxml.html as lh
from sklearn.decomposition import PCA
from sklearn import preprocessing
%matplotlib inline
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
import os
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.neural_network import MLPClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from operator import itemgetter
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Get Data

List of NBA Teams

In [ ]:
TEAMS_ID_NAME=[['1610612737', 'ATL'],
 ['1610612738', 'BOS'],
 ['1610612741', 'CHI'],
 ['1610612766', 'CHR'],
 ['1610612739', 'CLE'],
 ['1610612742', 'DAL'],
 ['1610612743' ,'DEN'],
 ['1610612765' ,'DET'],
 ['1610612744', 'GSW'],
 ['1610612745' ,'HOU'],
 ['1610612754' ,'IND'],
 ['1610612746' ,'LAC'],
 ['1610612747' ,'LAL'],
 ['1610612763' ,'MEM'],
 ['1610612748' ,'MIA'],
 ['1610612749' ,'MIL'],
 ['1610612750' ,'MIN'],
 ['1610612751' ,'NJN'],
 ['1610612740' ,'NOH'],
 ['1610612752' ,'NYK'],
 ['1610612753' ,'ORL'],
 ['1610612755' ,'PHI'],
 ['1610612756' ,'PHO'],
 ['1610612757' ,'POR'],
 ['1610612758' ,'SAC'],
 ['1610612759' ,'SAS'],
 ['1610612760' ,'SEA'],
 ['1610612761' ,'TOR'],
 ['1610612762' ,'UTA'],
 ['1610612764' ,'WAS']]

In [ ]:
# List of team ID's
team_ID = [1610612737, 1610612738, 1610612741, 1610612766, 1610612739, 1610612742, 1610612743, 1610612765, 1610612744, 1610612745, 1610612754, 1610612746, 1610612747, 1610612763, 1610612748, 1610612749, 1610612750, 1610612751, 1610612740, 1610612752, 1610612753, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612764]


##Team Season Data

In [ ]:
d = os.path.join("/content/team_season.txt")
team_season = np.loadtxt(d, delimiter=',', dtype='str')
# get header
header = team_season[0]
# remove first row, header
team_season = team_season[1:]
# convert it to an CSV file
pd.DataFrame(team_season).to_csv("team_stats.csv", header = header,index=False)

In [ ]:
# list of features we want to take from the team season data
col_list = ["team", "year",'o_ftm','o_reb' ,'o_asts' ,'o_pf' ,'o_stl' ,'o_to' ,'o_blk', 'o_3pm','o_pts',
 'd_ftm' ,'d_reb', 'd_asts' ,'d_pf',
 'd_stl', 'd_to', 'd_blk','d_3pm','d_pts','won','lost']

team_stats = pd.read_csv("team_stats.csv",usecols=col_list)

#get team stats for 2004
stats_df = team_stats.loc[team_stats["year"] == 2004]
stats_df.head()

,team,year,o_ftm,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_pts,d_ftm,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_pts,won,lost
1157,ATL,2004,1417,3435,1614,2009,629,1319,344,304,7605,1846,3385,1803,1777,716,1235,484,467,8401,13,69
1158,BOS,2004,1775,3347,1810,2000,667,1297,423,437,8304,1825,3466,1859,1937,714,1248,402,488,8233,45,37
1159,CHI,2004,1536,3592,1743,2028,576,1371,369,511,7745,1716,3503,1715,1914,685,1272,434,416,7658,47,35
1160,CHR,2004,1487,3418,1794,1893,695,1192,440,320,7729,1748,3545,1768,1857,624,1304,446,442,8220,18,64
1161,CLE,2004,1634,3469,1851,1850,654,1141,461,300,7914,1601,3262,1737,1770,549,1226,419,422,7849,42,40


In [ ]:
TeamStats_2004 = np.array(stats_df)
#skip first 2 columns (team name and year)
team_stats = TeamStats_2004[:,2 :] 

## Game Data

In [ ]:
col_list =["HOME_TEAM_ID","VISITOR_TEAM_ID","SEASON", "HOME_TEAM_WINS"]
game_data = pd.read_csv("games.csv", usecols=col_list)
# get games only for season 2005
game_data = game_data.loc[game_data["SEASON"] == 2005]
game_data.head()

,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS
11654,1610612742,1610612748,2005,0
11655,1610612748,1610612742,2005,1
11656,1610612748,1610612742,2005,1
11657,1610612748,1610612742,2005,1
11658,1610612742,1610612748,2005,1


In [ ]:
# convert into a np array
game_data = np.array(game_data)

# Create the Feature Vector

In [ ]:
def createDataSet(game_data,usePCA,isNormalized):
  pca = PCA(n_components=18)
  normalizer = preprocessing.MinMaxScaler()
  data = []
  for i in range(len(game_data)):

    # find location home_team stats
    find_index = np.where(team_ID == game_data[i][0])
    index = find_index[0][0]
    # get home_team stats
    home_side = np.array(team_stats[index])

  
    # get away_team stats
    find_index = np.where(team_ID == game_data[i][1])
    index = find_index[0][0]
    away_side =  np.array(team_stats[index])


    # create feature vector concatenate home_team stats and away_team stats
    feature_vector = np.concatenate((home_side, away_side), axis=None)

    
    # add fearure vector to data
    data.append(feature_vector)

  #NORMALIZE
  if (isNormalized):
    data = normalizer.fit_transform(data)

  #reduce feature vector using PCA
  if (usePCA):
    data = pca.fit_transform(data)

  ndata = []

  for z in range(len(game_data)):
    result = []
    result.append(game_data[z][3])

    # create feature vector by appending the outcome labled result of the game  
    feature_vector = np.concatenate((data[z],result), axis=None)
    ndata.append(feature_vector)

  data = np.array(ndata)
  
  return data

# ML Techniques

Scoring matrix 

In [ ]:
def scores(model):
    
    model.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    
    print("Accuracy score: %.3f" % metrics.accuracy_score(ytest, y_pred))
    print("Recall: %.3f" % metrics.recall_score(ytest, y_pred))
    print("Precision: %.3f" % metrics.precision_score(ytest, y_pred))
    print("F1: %.3f" % metrics.f1_score(ytest, y_pred))
    
    proba = model.predict_proba(xtest)
    print("Log loss: %.3f" % metrics.log_loss(ytest, proba))

    pos_prob = proba[:, 1]
    print("Area under ROC curve: %.3f" % metrics.roc_auc_score(ytest, pos_prob))
    
    cv = cross_val_score(model, xtest, ytest, cv = 10, scoring = 'accuracy')
    print("Accuracy (cross validation score): %0.3f (+/- %0.3f)" % (cv.mean(), cv.std() * 2))

    print("confusion matrix")
    print( metrics.confusion_matrix(ytest, y_pred) )
    
    return y_pred

## Training and Testing our Models

Get the Different Data Sets

In [ ]:
#Original data
dA = createDataSet(game_data,False,False)
#PCA with Normalized Data
dB = createDataSet(game_data,True,True)
#PCA Data
dC = createDataSet(game_data,True,False)
#Normalized Data
dD = createDataSet(game_data,False,True)

**Results using Original Data (NO PCA or Normalization)**

In [ ]:
trainA, testA = train_test_split( dA, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 
print("Training set size: %.0f" % len(xtrain))
print("Testing set size: %.0f" % len(xtest))
print()


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',C=100,probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 18, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

Training set size: 1074
Testing set size: 358

SVM STATS
Accuracy score: 0.634
Recall: 0.877
Precision: 0.649
F1: 0.746
Log loss: 0.629
Area under ROC curve: 0.666
Accuracy (cross validation score): 0.645 (+/- 0.129)
confusion matrix
[[ 35 104]
 [ 27 192]]

knn STATS
Accuracy score: 0.634
Recall: 0.744
Precision: 0.685
F1: 0.713
Log loss: 0.635
Area under ROC curve: 0.654
Accuracy (cross validation score): 0.642 (+/- 0.120)
confusion matrix
[[ 64  75]
 [ 56 163]]

logreg STATS
Accuracy score: 0.654
Recall: 0.804
Precision: 0.685
F1: 0.739
Log loss: 0.616
Area under ROC curve: 0.685
Accuracy (cross validation score): 0.625 (+/- 0.110)
confusion matrix
[[ 58  81]
 [ 43 176]]


**Results using PCA Data**

In [ ]:
trainA, testA = train_test_split( dC, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 18, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

SVM STATS
Accuracy score: 0.656
Recall: 0.831
Precision: 0.679
F1: 0.747
Log loss: 0.625
Area under ROC curve: 0.680
Accuracy (cross validation score): 0.640 (+/- 0.183)
confusion matrix
[[ 53  86]
 [ 37 182]]

knn STATS
Accuracy score: 0.656
Recall: 0.763
Precision: 0.702
F1: 0.731
Log loss: 0.630
Area under ROC curve: 0.662
Accuracy (cross validation score): 0.626 (+/- 0.136)
confusion matrix
[[ 68  71]
 [ 52 167]]

logreg STATS
Accuracy score: 0.628
Recall: 0.817
Precision: 0.658
F1: 0.729
Log loss: 0.627
Area under ROC curve: 0.669
Accuracy (cross validation score): 0.639 (+/- 0.201)
confusion matrix
[[ 46  93]
 [ 40 179]]


**Results using Normalized Data**

In [ ]:
trainA, testA = train_test_split( dD, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',C=100,probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 18, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

SVM STATS
Accuracy score: 0.561
Recall: 0.598
Precision: 0.655
F1: 0.625
Log loss: 0.661
Area under ROC curve: 0.585
Accuracy (cross validation score): 0.629 (+/- 0.172)
confusion matrix
[[ 70  69]
 [ 88 131]]

knn STATS
Accuracy score: 0.651
Recall: 0.749
Precision: 0.701
F1: 0.724
Log loss: 0.626
Area under ROC curve: 0.676
Accuracy (cross validation score): 0.645 (+/- 0.082)
confusion matrix
[[ 69  70]
 [ 55 164]]

logreg STATS
Accuracy score: 0.634
Recall: 0.817
Precision: 0.663
F1: 0.732
Log loss: 0.616
Area under ROC curve: 0.684
Accuracy (cross validation score): 0.645 (+/- 0.198)
confusion matrix
[[ 48  91]
 [ 40 179]]


**Results using PCA with Normalized Data**

In [ ]:
trainA, testA = train_test_split( dB, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 18, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

SVM STATS
Accuracy score: 0.651
Recall: 0.817
Precision: 0.678
F1: 0.741
Log loss: 0.629
Area under ROC curve: 0.674
Accuracy (cross validation score): 0.639 (+/- 0.114)
confusion matrix
[[ 54  85]
 [ 40 179]]

knn STATS
Accuracy score: 0.645
Recall: 0.753
Precision: 0.693
F1: 0.722
Log loss: 0.628
Area under ROC curve: 0.669
Accuracy (cross validation score): 0.659 (+/- 0.126)
confusion matrix
[[ 66  73]
 [ 54 165]]

logreg STATS
Accuracy score: 0.631
Recall: 0.817
Precision: 0.661
F1: 0.731
Log loss: 0.627
Area under ROC curve: 0.671
Accuracy (cross validation score): 0.634 (+/- 0.176)
confusion matrix
[[ 47  92]
 [ 40 179]]
